Задание 2
Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.
Сделайте агрумент n_estimators равным 1000,
max_depth должен быть равен 12 и random_state сделайте равным 42.
Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression,
но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0],
чтобы получить из датафрейма одномерный массив Numpy,
так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.
Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
Напишите в комментариях к коду, какая модель в данном случае работает лучше.


In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_boston

In [3]:
boston = load_boston()

C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset

In [4]:
data = boston.data

In [5]:
feature_names = boston.feature_names

In [6]:
X = pd.DataFrame(data, columns = feature_names)

In [7]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [8]:
target = boston.target

In [9]:
y = pd.DataFrame(target, columns = ['Price'])

In [10]:
y.head()

,Price
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [13]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
parameters = [{'n_estimators': [1000],
              'max_features': [9],
              'max_depth': [12]}]

In [16]:
model = GridSearchCV(estimator = RandomForestRegressor(random_state = 42),
                  param_grid = parameters,
                  scoring = 'accuracy',
                  cv = 5)

In [17]:
model.fit(X_train, y_train.values[:, 0])

C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\Users\Marusai\anaconda3\lib\site-packages\sklearn\metrics\_classification.py", line 104, in _check_targe

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_depth': [12], 'max_features': [9],
                          'n_estimators': [1000]}],
             scoring='accuracy')

In [18]:
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, model.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

nan (+/-nan) for {'max_depth': 12, 'max_features': 9, 'n_estimators': 1000}


In [19]:
from sklearn.metrics import r2_score

In [20]:
y_pred = model.predict(X_test)

In [21]:
check_test = pd.DataFrame({'y_test': y_test['Price'],
                          'y_pred': y_pred.flatten()},
                         columns = ['y_test', 'y_pred'])

In [22]:
check_test.head(10)

,y_test,y_pred
173,23.6,22.865454
274,32.4,31.386130
491,13.6,16.179176
72,22.8,23.737240
452,16.1,17.245431
76,20.0,21.933830
316,17.8,19.526287
140,14.0,14.758857
471,19.6,21.156621
500,16.8,20.889086


In [23]:
r2_score(y_test, y_pred)

0.8755328551767853

In [24]:
# Модель model работает лучше, чем модель lr, т.к. r2_score у model выше и ближе к единице.